In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
yazi = "ahmet"

In [3]:
vocab = set(yazi)
vocab

{'a', 'e', 'h', 'm', 't'}

In [4]:
vocab2= sorted(list(vocab))
vocab2

['a', 'e', 'h', 'm', 't']

In [5]:
for i in enumerate(vocab2):
    print(i)

(0, 'a')
(1, 'e')
(2, 'h')
(3, 'm')
(4, 't')


In [6]:
abc = {karakter:id for id, karakter in enumerate(vocab2)}
print(abc)

{'a': 0, 'e': 1, 'h': 2, 'm': 3, 't': 4}


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [8]:
from pathlib import Path
text = Path("tiny-shakespeare.txt").read_text()
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [50]:
class CharTokenizer:
    def __init__(self,vocabulary):
        self.token_id_for_char = {char:token_id for token_id, char in enumerate(vocabulary)}
        self.char_for_token_id = {token_id:char for token_id, char in enumerate(vocabulary)}

    @staticmethod
    def train_from_text(text):
        vocabulary = set(text)
        return CharTokenizer(sorted(list(vocabulary)))

    def encode(self,text):
        token_ids = []
        for char in text:
            token_ids.append(self.token_id_for_char[char])
        return torch.tensor(token_ids,dtype=torch.long)

    def decode(self,token_ids):
        chars =[]
        for id in token_ids.tolist():
            chars.append(self.char_for_token_id[id])
        return ''.join(chars)

    def vocabulary_size(self):
        return len(self.token_id_for_char)

In [51]:
tokenizer = CharTokenizer.train_from_text(text)

In [52]:
print(tokenizer.encode("Hello world"))

tensor([20, 43, 50, 50, 53,  1, 61, 53, 56, 50, 42])


In [53]:
print(tokenizer.encode("Ahmet"))

tensor([13, 46, 51, 43, 58])


In [54]:
tokenized_text01 = tokenizer.encode("Hello world Ahmet!!")
print(tokenizer.decode(tokenized_text01))

Hello world Ahmet!!


In [55]:
import pprint
pp = pprint.PrettyPrinter(depth=4)
pp.pprint(tokenizer.char_for_token_id)
pp.pprint(tokenizer.token_id_for_char)

{0: '\n',
 1: ' ',
 2: '!',
 3: '$',
 4: '&',
 5: "'",
 6: ',',
 7: '-',
 8: '.',
 9: '3',
 10: ':',
 11: ';',
 12: '?',
 13: 'A',
 14: 'B',
 15: 'C',
 16: 'D',
 17: 'E',
 18: 'F',
 19: 'G',
 20: 'H',
 21: 'I',
 22: 'J',
 23: 'K',
 24: 'L',
 25: 'M',
 26: 'N',
 27: 'O',
 28: 'P',
 29: 'Q',
 30: 'R',
 31: 'S',
 32: 'T',
 33: 'U',
 34: 'V',
 35: 'W',
 36: 'X',
 37: 'Y',
 38: 'Z',
 39: 'a',
 40: 'b',
 41: 'c',
 42: 'd',
 43: 'e',
 44: 'f',
 45: 'g',
 46: 'h',
 47: 'i',
 48: 'j',
 49: 'k',
 50: 'l',
 51: 'm',
 52: 'n',
 53: 'o',
 54: 'p',
 55: 'q',
 56: 'r',
 57: 's',
 58: 't',
 59: 'u',
 60: 'v',
 61: 'w',
 62: 'x',
 63: 'y',
 64: 'z'}
{'\n': 0,
 ' ': 1,
 '!': 2,
 '$': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '3': 9,
 ':': 10,
 ';': 11,
 '?': 12,
 'A': 13,
 'B': 14,
 'C': 15,
 'D': 16,
 'E': 17,
 'F': 18,
 'G': 19,
 'H': 20,
 'I': 21,
 'J': 22,
 'K': 23,
 'L': 24,
 'M': 25,
 'N': 26,
 'O': 27,
 'P': 28,
 'Q': 29,
 'R': 30,
 'S': 31,
 'T': 32,
 'U': 33,
 'V': 34,
 'W': 35,
 'X': 36

In [56]:
from torch.utils.data import Dataset


class TokenIdsDataset(Dataset):
    def __init__(self,data,block_size):
        self.data = data
        self.block_size = block_size

    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, pos):
        assert pos < len(self.data) - self.block_size

        x = self.data[pos:pos+self.block_size]
        y = self.data[pos+1:pos+1+self.block_size]

        return x,y

In [57]:
tokenized_text = tokenizer.encode(text)
tokenized_text

tensor([18, 47, 56,  ..., 45,  8,  0])

In [58]:
len(tokenized_text)

1115394

In [59]:
dataset = TokenIdsDataset(tokenized_text,block_size=64)

In [60]:
x,y = dataset[0]

In [61]:
x

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50])

In [62]:
y

tensor([47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44, 53,
        56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,  1,
        44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1, 57,
        54, 43, 39, 49,  8,  0,  0, 13, 50, 50])

In [63]:
tokenizer.decode(x)

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAl'

In [64]:
tokenizer.decode(y)

'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll'

In [65]:
from torch.utils.data import DataLoader , RandomSampler

sampler = RandomSampler(dataset,replacement=True)
dataloader = DataLoader(dataset,batch_size=2,sampler=sampler)

In [66]:
x,y = next(iter(dataloader))

In [67]:
x.shape

torch.Size([2, 64])

In [68]:
x

tensor([[57, 43, 52, 45, 43, 56,  1, 61, 46, 53,  1, 40, 47, 42, 57,  1, 40, 43,
         61, 39, 56, 43,  0, 27, 44,  1, 61, 46, 39, 58,  1, 47, 57,  1, 58, 53,
          1, 40, 43,  1, 42, 56, 43, 39, 42, 43, 42,  8,  0,  0, 31, 21, 15, 21,
         26, 21, 33, 31, 10,  0, 32, 43, 50, 50],
        [50, 47, 41, 43,  8,  0,  0, 30, 21, 34, 17, 30, 31, 10,  0, 31, 39, 61,
          1, 63, 53, 59,  1, 58, 46, 43,  1, 49, 47, 52, 45,  1, 58, 53,  7, 42,
         39, 63,  6,  1, 51, 63,  1, 24, 53, 56, 42,  1, 53, 44,  1, 16, 43, 56,
         40, 63, 12,  0,  0, 16, 17, 30, 14, 37]])

In [69]:
tokenizer.decode(x[0])

'senger who bids beware\nOf what is to be dreaded.\n\nSICINIUS:\nTell'

In [70]:
tokenizer.decode(y[0])

'enger who bids beware\nOf what is to be dreaded.\n\nSICINIUS:\nTell '

In [71]:
## EMBEDDINGS

In [72]:
import torch
import torch.nn as nn
vocab_size = 10000
embedding_dim = 75

In [73]:
embedding = nn.Embedding(num_embeddings=vocab_size,embedding_dim=embedding_dim)

In [74]:
token_ids = torch.LongTensor([11,9783,376,45])

In [75]:
embedded = embedding(token_ids)

In [76]:
embedded.shape

torch.Size([4, 75])

In [77]:
config = {
    "vocabulary_size": tokenizer.vocabulary_size(),
    "context_size": 256,
    "embedding_dim": 768,
    "heads_num": 12,
    "layers_num": 10,
    "dropout_rate": 0.1,
    "use_bias": False,
}
config["head_size"] = config["embedding_dim"] // config["heads_num"]

In [78]:
class AttentionHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.Q_weights = nn.Linear(config["embedding_dim"], config["head_size"], config["use_bias"])
        self.K_weights = nn.Linear(config["embedding_dim"], config["head_size"], config["use_bias"])
        self.V_weights = nn.Linear(config["embedding_dim"], config["head_size"], config["use_bias"])
        self.dropout = nn.Dropout(config["dropout_rate"])
        casual_attention_mask = torch.tril(torch.ones(config["context_size"], config["context_size"]))
        self.register_buffer('casual_attention_mask', casual_attention_mask)

    def forward(self, input):
        batch_size, tokens_num, embedding_dim = input.shape
        Q = self.Q_weights(input)
        K = self.K_weights(input)
        V = self.V_weights(input)
        attention_scores = Q @ K.transpose(1, 2)
        attention_scores = attention_scores.masked_fill(
            self.casual_attention_mask[:tokens_num,:tokens_num] == 0,
            -torch.inf
        )
        attention_scores = attention_scores / ( K.shape[-1] ** 0.5 )
        attention_scores = torch.softmax(attention_scores, dim=-1)
        attention_scores = self.dropout(attention_scores)
        return attention_scores @ V

In [79]:
input = torch.rand(8, config["context_size"], config["embedding_dim"])
ah = AttentionHead(config)
output = ah(input)
print(output.shape)  # Expected output: torch.Size([8, 256, 64])

torch.Size([8, 256, 64])


In [80]:
class MultiHeadAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        heads_list = [AttentionHead(config) for _ in range(config["heads_num"])]
        self.heads = nn.ModuleList(heads_list)
        self.linear = nn.Linear(config["embedding_dim"], config["embedding_dim"])
        self.dropout = nn.Dropout(config["dropout_rate"])

    def forward(self, input):
        heads_outputs = [head(input) for head in self.heads]
        scores_change = torch.cat(heads_outputs, dim=-1)
        scores_change = self.linear(scores_change)
        return self.dropout(scores_change)

In [81]:
mha = MultiHeadAttention(config)
output = mha(input)
print(output.shape)  # Expected output: torch.Size([8, 256, 768])

torch.Size([8, 256, 768])


In [82]:
class FeedForward(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.linear_layers = nn.Sequential(
            nn.Linear(config["embedding_dim"], config["embedding_dim"] * 4),
            nn.GELU(),
            nn.Linear(config["embedding_dim"] * 4, config["embedding_dim"]),
            nn.Dropout(config["dropout_rate"])
        )

    def forward(self, input):
        return self.linear_layers(input)

In [83]:
ff = FeedForward(config)
input = torch.rand(8, config["context_size"], config["embedding_dim"])
output = ff(input)
print(output.shape)  # Expected: torch.Size([8, 256, 768])

torch.Size([8, 256, 768])


In [84]:
class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.multi_head = MultiHeadAttention(config)
        self.layer_norm_1 = nn.LayerNorm(config["embedding_dim"])
        self.feed_forward = FeedForward(config)
        self.layer_norm_2 = nn.LayerNorm(config["embedding_dim"])

    def forward(self, input):
        residual = input
        x = self.multi_head(self.layer_norm_1(input))
        x = x + residual

        residual = x
        x = self.feed_forward(self.layer_norm_2(x))
        return x + residual

In [85]:
b = Block(config)
input = torch.rand(8, config["context_size"], config["embedding_dim"])
output = b(input)
print(output.shape)  # Expected: torch.Size([8, 256, 768])

torch.Size([8, 256, 768])


In [86]:
class DemoGPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.token_embedding_layer = nn.Embedding(config["vocabulary_size"], config["embedding_dim"])
        self.positional_embedding_layer = nn.Embedding(config["context_size"], config["embedding_dim"])
        blocks = [Block(config) for _ in range(config["layers_num"])]
        self.layers = nn.Sequential(*blocks)
        self.layer_norm = nn.LayerNorm(config["embedding_dim"])
        self.unembedding = nn.Linear(config["embedding_dim"], config["vocabulary_size"], bias=False)

    def forward(self, token_ids):
        batch_size, tokens_num = token_ids.shape
        x = self.token_embedding_layer(token_ids)
        sequence = torch.arange(tokens_num, device=device)
        x = x + self.positional_embedding_layer(sequence)
        x = self.layers(x)
        x = self.layer_norm(x)
        x = self.unembedding(x)
        return x

In [87]:
model = DemoGPT(config).to(device)
output = model(tokenizer.encode("Hi").unsqueeze(dim=0).to(device))
print(output.shape)  # Expected: torch.Size([1, 2, 65])

torch.Size([1, 2, 65])


In [88]:
def generate(model, prompt_ids, max_tokens):
    output_ids = prompt_ids
    for _ in range(max_tokens):
        if output_ids.shape[1] >= config["context_size"]:
            break
        with torch.no_grad():
            logits = model(output_ids)
        logits = logits[:, -1, :]
        probs = F.softmax(logits, dim=-1)
        next_token_id = torch.multinomial(probs, num_samples=1)
        output_ids = torch.cat([output_ids, next_token_id], dim=-1)
    return output_ids

In [89]:
def generate_with_prompt(model, tokenizer, prompt, max_tokens=100):
    model.eval()
    prompt = tokenizer.encode(prompt).unsqueeze(dim=0).to(device)
    return tokenizer.decode(generate(model, prompt, max_tokens=max_tokens)[0])

In [90]:
generate_with_prompt(model, tokenizer, "First Citizen:\n")
# Output: "First Citizen:\nDc?SlGAOau$r'QE'us3:nSfYjnkJpFkLVEc$GovIKqlQHxRa\nA;MWyCrmtZkMp?jCiqNPWFutrO $YcaR:rb!BdGtPyvE$-PVVsE"

'First Citizen:\nUyJuEauJBsJb?DyjVl3nxKs GulNK;IpuS?ogqmBBqfTVSvkTVbVftXfZh?koauORHEs,MzSbEhTd.eYNdYMkKTdN&EgLOaO?-YL'

In [92]:
tokenized_text = tokenizer.encode(text).to(device)

In [106]:
batch_size = 64
train_iterations = 5000
evaluation_interval = 100
learning_rate = 4e-4
train_split = 0.9

In [107]:
tokenized_text = tokenizer.encode(text).to(device)
train_count = int(train_split * len(tokenized_text))
train_data,validation_data = tokenized_text[:train_count], tokenized_text[train_count:]

In [108]:
train_dataset = TokenIdsDataset(train_data, config["context_size"])
validation_dataset = TokenIdsDataset(validation_data, config["context_size"])

In [109]:
from torch.utils.data import Dataset, DataLoader, RandomSampler

train_sampler = RandomSampler(train_dataset, num_samples=batch_size * train_iterations, replacement=True)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)

validation_sampler = RandomSampler(validation_dataset,replacement=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, sampler=validation_sampler)

In [110]:
@torch.no_grad()
def calculate_validation_loss(model, batches_num):
    model.eval()
    total_loss = 0

    validation_iter = iter(validation_dataloader)

    for _ in range(batches_num):
        input, targets = next(validation_iter)
        logits = model(input)

        logits_view = logits.view(batch_size * config["context_size"], config["vocabulary_size"])
        targets_view = targets.view(batch_size * config["context_size"])

        loss = F.cross_entropy(logits_view, targets_view)

        total_loss += loss.item()

    average_loss = total_loss / batches_num

    return average_loss

In [111]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [98]:
config = {
    "vocabulary_size": tokenizer.vocabulary_size(),
    "context_size": 256,
    "embedding_dim": 768,
    "heads_num": 12,
    "layers_num": 10,
    "dropout_rate": 0.1,
    "use_bias": False,
}
config["head_size"] = config["embedding_dim"] // config["heads_num"]

In [112]:
import os
from IPython.display import display,clear_output
from matplotlib import pyplot as plt
from IPython.display import display
import ipywidgets as widgets
%matplotlib inline

In [113]:
plot_output = widgets.Output()
display(plot_output)

def update_plot(train_losses, train_steps, validation_losses, validation_steps):
    with plot_output:
        clear_output(wait=True)
        plt.figure(figsize=(7, 5))
        plt.plot(train_steps, train_losses, label='Training Loss')
        plt.plot(validation_steps, validation_losses, label='Validation Loss')
        plt.title('Training and Validation Loss')
        plt.xlabel('epoch')
        plt.legend(loc='center left')
        plt.grid(True)
        plt.show()

train_losses = []
train_steps = []
eval_losses = []
eval_steps = []

for step_num, sample in enumerate(train_dataloader):

    model.train()
    input, targets = sample
    logits = model(input)

    logits_view = logits.view(batch_size * config["context_size"], config["vocabulary_size"])
    targets_view = targets.view(batch_size * config["context_size"])

    loss = F.cross_entropy(logits_view, targets_view)
    # Backward propagation
    loss.backward()
    # Update model parameters
    optimizer.step()
    # Set to None to reduce memory usage
    optimizer.zero_grad(set_to_none=True)

    train_losses.append((loss.item()))
    train_steps.append(step_num)

    print(f"Step {step_num}. Loss {loss.item():.3f}")

    if step_num % evaluation_interval == 0:
        print("Demo GPT:\n" + generate_with_prompt(model, tokenizer, "\n"))
        validation_loss = calculate_validation_loss(model,batches_num=10)
        eval_losses.append(validation_loss)
        eval_steps.append(step_num)

        print(f"Step {step_num}. Validation loss: {validation_loss:.3f}")

    update_plot(train_losses,train_steps,eval_losses,eval_steps)

Output()

Step 0. Loss 2.998
Demo GPT:

s llG st ssms   lA v l h f shs s s ll s  ls tn f fc   t  s ty nd s s sps s l  s d k s     s h s s d 
Step 0. Validation loss: 4.019
Step 1. Loss 3.984
Step 2. Loss 3.741
Step 3. Loss 3.518
Step 4. Loss 3.281
Step 5. Loss 3.155
Step 6. Loss 3.058
Step 7. Loss 3.090
Step 8. Loss 2.941
Step 9. Loss 2.848
Step 10. Loss 2.827
Step 11. Loss 2.793
Step 12. Loss 2.769
Step 13. Loss 2.726
Step 14. Loss 2.698
Step 15. Loss 2.688
Step 16. Loss 2.643
Step 17. Loss 2.658
Step 18. Loss 2.641
Step 19. Loss 2.622
Step 20. Loss 2.610
Step 21. Loss 2.605
Step 22. Loss 2.586
Step 23. Loss 2.588
Step 24. Loss 2.585
Step 25. Loss 2.574
Step 26. Loss 2.545
Step 27. Loss 2.565
Step 28. Loss 2.564
Step 29. Loss 2.548
Step 30. Loss 2.547
Step 31. Loss 2.547
Step 32. Loss 2.532
Step 33. Loss 2.539
Step 34. Loss 2.505
Step 35. Loss 2.523
Step 36. Loss 2.524
Step 37. Loss 2.526
Step 38. Loss 2.535
Step 39. Loss 2.506
Step 40. Loss 2.490
Step 41. Loss 2.495
Step 42. Loss 2.520
Step 43

KeyboardInterrupt: 

In [30]:
prompt = "During the latest presentation OpenAI"
# prompt = "During the latest presentation Apple has announced"
model = "openai-community/gpt2-large"

In [31]:
import torch
torch.cuda.is_available()

True

In [34]:
from transformers import pipeline

# Load the text generation pipeline
text_generator = pipeline("text-generation", model=model,device=0)

In [35]:
# Generate text
generated_texts = text_generator(prompt, max_length=100, truncation=True, num_return_sequences=1)

print(generated_texts[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
C:\Users\ahmet.yaylalioglu\.conda\envs\sam2\Lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:544: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


During the latest presentation OpenAI CEO and co-founder Demis Hassabis took to the stage again to introduce the new project, which is the culmination of years of thinking about and planning. Hassabis said that the two main goals for OpenAI at this time are "to create a platform that gives rise to a new class of AI researchers, which then leads to creating AI that can actually solve real problems" and "to empower AI in the public sphere with the first step toward creating more secure AI


In [36]:
for do_sample in [
    False, # Greedy Search
    True   # Multinomial sampling
]:
    generated_texts = text_generator(prompt, max_length=100, num_return_sequences=1, do_sample=do_sample, num_beams=1)

    print("-----------------------------------")
    print("Parameters:")
    print("-----------------------------------")
    print(f"do_sample={do_sample}")
    print("-----------------------------------")
    print("Generation:")
    print("-----------------------------------")
    print(generated_texts[0]['generated_text'])
    print("-----------------------------------")
    print("\n\n")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-----------------------------------
Parameters:
-----------------------------------
do_sample=False
-----------------------------------
Generation:
-----------------------------------
During the latest presentation OpenAI's CEO, John Robb, said that the company is working on a "biggest AI project ever" and that it is "going to be a big deal."

Robb also said that the company is working on a "biggest AI project ever" and that it is "going to be a big deal."

"We're going to be a big deal," he said. "We're going to be the biggest AI project ever."


-----------------------------------



-----------------------------------
Parameters:
-----------------------------------
do_sample=True
-----------------------------------
Generation:
-----------------------------------
During the latest presentation OpenAI's AI chief and former Google DeepMind CEO Demis Hassabis outlined the future of AI with its latest "AlphaGo" artificial general intelligence software and argued against the notion of hum

In [37]:
# Beam-search strategy

for beams in [1, 2, 4, 8]:
    generated_texts = text_generator(prompt, max_length=100, num_return_sequences=1, do_sample=False, num_beams=beams)

    print("-----------------------------------")
    print("Parameters:")
    print("-----------------------------------")
    print(f"num_beams={beams}")
    print("-----------------------------------")
    print("Generation:")
    print("-----------------------------------")
    print(generated_texts[0]['generated_text'])
    print("-----------------------------------")
    print("\n\n")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-----------------------------------
Parameters:
-----------------------------------
num_beams=1
-----------------------------------
Generation:
-----------------------------------
During the latest presentation OpenAI's CEO, John Robb, said that the company is working on a "biggest AI project ever" and that it is "going to be a big deal."

Robb also said that the company is working on a "biggest AI project ever" and that it is "going to be a big deal."

"We're going to be a big deal," he said. "We're going to be the biggest AI project ever."


-----------------------------------





Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-----------------------------------
Parameters:
-----------------------------------
num_beams=2
-----------------------------------
Generation:
-----------------------------------
During the latest presentation OpenAI's CEO and co-founder Demis Hassabis said that the company is working on a new type of artificial intelligence that will be able to learn from its mistakes.

"We're working on a new type of artificial intelligence that will learn from its mistakes," Hassabis said. "We're not going to be able to predict the future, but we're going to be able to learn from our mistakes."

Hassabis said that the company is working
-----------------------------------





Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-----------------------------------
Parameters:
-----------------------------------
num_beams=4
-----------------------------------
Generation:
-----------------------------------
During the latest presentation OpenAI's CEO and co-founder Demis Hassabis said, "We are excited to announce that we have reached an agreement with DeepMind to work together on the development of the next generation of artificial intelligence."

DeepMind is a British artificial intelligence company founded in 2012. It is best known for its work on AlphaGo, a computer program that defeated the world's best Go player, Lee Sedol, at the end of 2016.

"We are excited
-----------------------------------



-----------------------------------
Parameters:
-----------------------------------
num_beams=8
-----------------------------------
Generation:
-----------------------------------
During the latest presentation OpenAI's CEO and co-founder Demis Hassabis said:

"We've been working on this for a long time. We've be

In [38]:
# Beam-search multinomial sampling

for beams in [1, 2, 4, 8]:
    generated_texts = text_generator(prompt, max_length=100, num_return_sequences=1, do_sample=True, num_beams=beams)

    print("-----------------------------------")
    print("Parameters:")
    print("-----------------------------------")
    print(f"num_beams={beams}")
    print("-----------------------------------")
    print("Generation:")
    print("-----------------------------------")
    print(generated_texts[0]['generated_text'])
    print("-----------------------------------")
    print("\n\n")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-----------------------------------
Parameters:
-----------------------------------
num_beams=1
-----------------------------------
Generation:
-----------------------------------
During the latest presentation OpenAI and others held yesterday to discuss their work involving a neural network and a convolutional layer of a super-deep neural network, they made several suggestions and criticisms of the work. In doing so, they showed two important things; First, that deep learning systems have gotten significantly better, compared to just a few years ago. Second, that today's most sophisticated techniques could indeed be taken beyond simple reinforcement learning and into more complex tasks, such as image recognition and text generation
-----------------------------------





Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-----------------------------------
Parameters:
-----------------------------------
num_beams=2
-----------------------------------
Generation:
-----------------------------------
During the latest presentation OpenAI, a San Francisco-based artificial intelligence company, showed off its latest project, DeepMind's AlphaGo, a computer program that beat the world's top Go player, Lee Sedol, in a six-game match in March.

AlphaGo's victory came after a series of complex games, in which the program learned to recognize patterns and play the game at a faster pace than humans.

"This is a tremendous achievement for DeepMind," said Deep
-----------------------------------





You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-----------------------------------
Parameters:
-----------------------------------
num_beams=4
-----------------------------------
Generation:
-----------------------------------
During the latest presentation OpenAI CEO Demis Hassabis revealed that the team at the company has been working on a new AI system that will be used in the game.

"We have been working on a new AI system, which will be used in the game," said Hassabis.

"We have been working on a new AI system, which will be used in the game," said Hassabis. "We have been working on a new AI system, which will be used in
-----------------------------------



-----------------------------------
Parameters:
-----------------------------------
num_beams=8
-----------------------------------
Generation:
-----------------------------------
During the latest presentation OpenAI CEO Demis Hassabis said, "This is the beginning of a new era for artificial intelligence, and we're excited to be a part of it."

OpenAI is a non-profit or

In [39]:
# Change the top-k parameter

for k in [1, 5, 10, 50, 100, 500]:
    generated_texts = text_generator(prompt, max_length=100, num_return_sequences=1, top_k=k)

    print("-----------------------------------")
    print("Parameters:")
    print("-----------------------------------")
    print(f"top_k={k}")
    print("-----------------------------------")
    print("Generation:")
    print("-----------------------------------")
    print(generated_texts[0]['generated_text'])
    print("-----------------------------------")
    print("\n\n")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-----------------------------------
Parameters:
-----------------------------------
top_k=1
-----------------------------------
Generation:
-----------------------------------
During the latest presentation OpenAI's CEO, John Robb, said that the company is working on a "biggest AI project ever" and that it is "going to be a big deal."

Robb also said that the company is working on a "biggest AI project ever" and that it is "going to be a big deal."

"We're going to be a big deal," he said. "We're going to be the biggest AI project ever."


-----------------------------------





Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-----------------------------------
Parameters:
-----------------------------------
top_k=5
-----------------------------------
Generation:
-----------------------------------
During the latest presentation OpenAI has shown the ability to create a self-driving car that can navigate around the streets of San Francisco. The self-driving cars have a range of about 30 miles, and they're being tested on public roads. It's a big improvement on the previous self-driving car that was shown in a video that OpenAI released in March. That video was shot by Google's self-driving car team, and it was a pretty impressive demonstration of what the team is capable
-----------------------------------





Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-----------------------------------
Parameters:
-----------------------------------
top_k=10
-----------------------------------
Generation:
-----------------------------------
During the latest presentation OpenAI had been working with a team of researchers from Google Brain to develop a computer-vision system that can detect objects at a distance, and that can recognize the shape and orientation of objects on the screen without having to know the object's shape. That is a huge step forward for the field.

This research will allow AI to make better decisions about objects on a scene. The system is able to recognize things such as objects with multiple faces, or objects that are very
-----------------------------------





Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-----------------------------------
Parameters:
-----------------------------------
top_k=50
-----------------------------------
Generation:
-----------------------------------
During the latest presentation OpenAI is hosting, a young computer scientist by the name of Ilya Sutskever, will be answering questions about the project. A similar video about the project is expected to be posted soon, but if you haven't already seen it, head over to the youtube.com/openai channel and watch. It's a pretty good look at what the team is actually making.

All in all, it's good to see that they're not really holding us
-----------------------------------





Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-----------------------------------
Parameters:
-----------------------------------
top_k=100
-----------------------------------
Generation:
-----------------------------------
During the latest presentation OpenAI has demonstrated using a three human brains encased in a neural network to perform AI tasks, such as understanding speech and playing music.

The neural network would then attempt to guess the theme song that would be played next and then automatically play it. Of course, the goal isn't to beat humans at an art contest, but to make AI machines that can perform meaningful tasks.

The whole thing is a little difficult and involves lots of images showing "mice
-----------------------------------



-----------------------------------
Parameters:
-----------------------------------
top_k=500
-----------------------------------
Generation:
-----------------------------------
During the latest presentation OpenAI, where each speaker contributes to his own computer (making not thi

In [40]:
# Change temperature

for temp in [0.1, 1.0, 2.0, 3.0]:
    generated_texts = text_generator(prompt, max_length=100, num_return_sequences=1, temperature=temp)

    print("-----------------------------------")
    print("Parameters:")
    print("-----------------------------------")
    print(f"temperature={temp}")
    print("-----------------------------------")
    print("Generation:")
    print("-----------------------------------")
    print(generated_texts[0]['generated_text'])
    print("-----------------------------------")
    print("\n\n")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-----------------------------------
Parameters:
-----------------------------------
temperature=0.1
-----------------------------------
Generation:
-----------------------------------
During the latest presentation OpenAI's CEO, John Robb, said that the company had been working on the project for a year and a half. He also said that the company had been working on the project for a year and a half.

Robb also said that the company had been working on the project for a year and a half. He also said that the company had been working on the project for a year and a half.

"We have been working on this project for
-----------------------------------





Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-----------------------------------
Parameters:
-----------------------------------
temperature=1.0
-----------------------------------
Generation:
-----------------------------------
During the latest presentation OpenAI and Facebook announced that they have teamed up and that OpenAI software will be used in the development of DeepDream, Facebook's artificial intelligence software that can create original 3D images of the environment.

That's very big news and very exciting news. I was told by OpenAI's founder, Yishai Stein, that it was the biggest breakthrough in deep learning since the inception of the company in 2011.

While that's not entirely the case, we
-----------------------------------





Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-----------------------------------
Parameters:
-----------------------------------
temperature=2.0
-----------------------------------
Generation:
-----------------------------------
During the latest presentation OpenAI sent for an invitation, a slide made out to an upcoming article about deep learning and computer vision revealed the future of VR. Not on the HTC HMD they are also making it possible for people living in future times -- even tomorrow time -- to see what happens after events happened and even live events on this device like Super Bowl 51 or The Simpsons finale. If your kids want to join VR adventures playing Mario Maker while hanging out in the backyard but in reality they just ran
-----------------------------------



-----------------------------------
Parameters:
-----------------------------------
temperature=3.0
-----------------------------------
Generation:
-----------------------------------
During the latest presentation OpenAI also explained a new kind of in